In [38]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client["phi4"]
collection = db["phi4:latest"]

# Fetch first 10 documents
docs = collection.find().limit(10)

In [42]:
databases = client.list_database_names()
databases

['admin', 'config', 'local', 'mistral', 'phi4']

In [3]:
docs = collection.find({}, {"ground_truth": 1, "answer": 1, "_id": 0})  # _id:0 excludes the default id field

In [4]:
results = [i for i in docs]

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(results)
df['pred'] = df['answer'].apply(lambda x: x.split('\n')[0])

In [7]:
df.head()

,ground_truth,answer,pred
0,True,True\n\nPersian (known as Farsi in Iran and Da...,True
1,True,True\n\nGood Samaritan laws are designed to pr...,True
2,True,True\n\nWindows Movie Maker is indeed part of ...,True
3,True,"True\n\nIn the context provided, confectioners...",True
4,False,False\n\nThe Elder Scrolls Online (ESO) and Sk...,False


In [8]:
def extract_label(answer):
    if answer.lower().startswith('true'):
        return True
    elif answer.lower().startswith('false'):
        return False
    else:
        return None

In [9]:
df['pred'] = df['answer'].apply(extract_label)

In [10]:
df['pred'] = df['pred'].fillna('UNK')

In [11]:
df['pred'].value_counts(dropna=False)

pred
True     5528
False    3816
UNK        83
Name: count, dtype: int64

In [12]:
pd.crosstab(df['pred'], df['ground_truth'])

ground_truth,False,True
pred,,
False,3119,697
True,410,5118
UNK,24,59


In [13]:
df.dtypes

ground_truth      bool
answer          object
pred            object
dtype: object

In [20]:
df['pred'].value_counts()

pred
True     5528
False    3816
UNK        83
Name: count, dtype: int64

In [30]:
df_filtered = df.copy()
df_filtered = df_filtered[df_filtered['pred'].isin([True, False])].copy()
df_filtered['pred'] = df_filtered['pred'].astype(bool)

In [31]:
df_filtered.dtypes

ground_truth      bool
answer          object
pred              bool
dtype: object

In [32]:
from sklearn.metrics import classification_report

In [33]:
df_filtered['ground_truth'].value_counts(dropna=False)

ground_truth
True     5815
False    3529
Name: count, dtype: int64

In [34]:
df_filtered['pred'].value_counts(dropna=False)

pred
True     5528
False    3816
Name: count, dtype: int64

In [35]:
# Generate report
report = classification_report(
    y_true=df_filtered['ground_truth'],
    y_pred=df_filtered['pred'],
    target_names=['False', 'True'],
    zero_division=0
)
print(report)

              precision    recall  f1-score   support

       False       0.82      0.88      0.85      3529
        True       0.93      0.88      0.90      5815

    accuracy                           0.88      9344
   macro avg       0.87      0.88      0.88      9344
weighted avg       0.88      0.88      0.88      9344

